In [ ]:
%pip install git+https://github.com/istat-methodology/semantic-search.git

# CIRCE vs Semantic Search - ATECO 2022

In [3]:
import pandas as pd
import numpy as np
import ast
import requests
import time
import json
from semantic_search.data import build_corpus
from semantic_search.local import LocalKnowledgeBase

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configs

In [14]:
EXTRACT_CIRCE : bool = False
CIRCE_URL     : str  = "https://www.istat.it/wp-content/themes/EGPbs5-child/ateco/atecor.php"
CIRCE_FILENAME: str  = "data/circe_results.json"
MERGE_DESCS   : bool = False

MODEL_ID : str = "BAAI/bge-m3"  # paraphrase-multilingual-MiniLM-L12-v2, LaBSE, Qwen/Qwen3-Embedding-0.6B

---

## Data

### Build the ATECO 2022 dataset (needed by semantic search algorithm)

In [ ]:
ateco_df = pd.read_csv("data/ateco_2022_raw.csv")
ateco_df = ateco_df[ateco_df["section"].str.len() == 8]

# These lists will be used to store the codes and texts in the final corpus
codes = []
texts = []

for i, row in ateco_df.iterrows():
    if row["title"]:
        codes.append(row["section"])
        texts.append(row["title"].lower())

    if type(row["description"]) == str:
        desc = row["description"]
        desc = desc.split(". Sono escluse")[0]
        desc_list = desc.split(" - ")
        for d in desc_list:
            codes.append(row["section"])
            texts.append(d.lower().strip("- "))

if MERGE_DESCS:
    ateco_df_merge = pd.DataFrame({
        "code": codes,
        "text": texts
    })
    ateco_df_merge = ateco_df_merge.groupby("code").aggregate({
        "text": lambda x: ".\n".join(x)
    })
    codes = ateco_df_merge.index.tolist()
    texts = ateco_df_merge["text"].tolist()

### Extract a sample of enterprise queries from ateco_sample_queries

In [6]:
sample_queries_df = pd.read_csv("data/ateco_sample_queries.csv", sep=";")

sampled_df = sample_queries_df.sample(1000, random_state=42)
sampled_df.drop(columns=["ID"], inplace=True)
sampled_df.rename(columns={"Stringa": "query"}, inplace=True)

# Remove extra spaces from queries
sampled_df["query"] = sampled_df["query"].str.replace(r"\s+", " ", regex=True).str.strip()

queries = sampled_df["query"].tolist()

### Build CIRCE labeled dataset.

In [15]:
if EXTRACT_CIRCE:
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    with open(CIRCE_FILENAME, "w") as f:
        json.dump({}, f)

    results_dict = {}
    for idx, query in enumerate(queries):
        results_dict[idx] = {}
        results_dict[idx]["query"] = query
        data = {"search": query}
        try:
            response = requests.post(CIRCE_URL, data=data, headers=headers)
            response_dict = ast.literal_eval(response.text.replace('""', '"'))

            results_dict[idx]["result"] = {}

            for j, res in enumerate(response_dict["0"]):
                results_dict[idx]["result"][j] = {}
                code = res["ateco_code"]
                desc = res["ateco_description"]
                results_dict[idx]["result"][j]["code"] = code
                results_dict[idx]["result"][j]["desc"] = desc

        except:
            results_dict[idx]["result"] = "ERROR"

        with open(CIRCE_FILENAME, "r") as f:
            json_data = json.load(f)
        json_data.update(results_dict)
        with open(CIRCE_FILENAME, "w") as f:
            json.dump(json_data, f)

        time.sleep(0.005)

else:
    with open(CIRCE_FILENAME, "r") as f:
        results_dict = json.load(f)

---

## Semantic Search
First, we create the knowledge base for the ATECO 2022 classification.

In [11]:
corpus = build_corpus(
    texts=texts,
    ids=list(range(len(texts))),
    metadata=[{"code": c} for c in codes]
)
base = LocalKnowledgeBase(
    corpus=corpus,
    model_id=MODEL_ID,
    batch_size=16
)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:18<00:00, 13.08it/s]


Then, we search the knowledge base and extract the matching between CIRCE and semantic search.

In [17]:
match_top_k = 10

results = base.search(queries, top_k=30)

def parse_retrieved(results):
    codes = [r.metadata["code"] for r in results]
    texts = [r.text for r in results]
    scores = [r.score for r in results]

    df = pd.DataFrame({
        "code": codes,
        "matched_text": texts,
        "score": scores
    })

    grouped_df = df.groupby("code").aggregate({
        "score": "max",
    }).reset_index()

    return grouped_df.sort_values("score", ascending=False)

queries_filtered = []
top_results = []
circe_guess = []
sem_search_guess = []

for idx, res in enumerate(results):
    circe = results_dict[str(idx)]["result"]
    if circe == "ERROR":
        continue
    ateco_circe = set(np.unique([circe[key]["code"] for key in circe.keys()]).tolist())

    sem_search_res = parse_retrieved(res.results).iloc[:match_top_k]["code"].tolist()
    sem_search_res = set([r[:-1] for r in sem_search_res])

    if ateco_circe.intersection(sem_search_res):
        top_results.append(1)
    else:
        top_results.append(0)

    circe_guess.append(list(ateco_circe))
    sem_search_guess.append(list(sem_search_res))
    queries_filtered.append(queries[idx])

print(f"Overlap (Top {match_top_k})): {round(np.mean(top_results)*100, 2)}%")

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 23.83it/s]


Overlap (Top 10)): 66.94%


Export the result of the analysis to a csv file.

In [ ]:
export_df = pd.DataFrame({
    "QUERY": queries_filtered,
    "CIRCE": circe_guess,
    "SEMANTIC": sem_search_guess,
    "OVERLAP": top_results
})

export_df.to_csv("analysis/circe_vs_semantic.csv", index=False, sep=";")